In [78]:
using Plots, Interact
gr()

Plots.GRBackend()

In [79]:
include("discs_in_box.jl")

dynamics

In [80]:
w, h = 1.5, 1
r = 0.24

@time times, positions, velocities, collision_types = dynamics(w, h, r, 1000000);

  2.901184 seconds (13.87 M allocations: 1006.389 MiB, 70.76% gc time)


In [81]:
showall(norm.(velocities)[1:100])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [82]:
showall(collision_types[1:100])

[-2, -3, 5, -3, 2, -2, 2, 1, -2, 5, 1, 4, -3, 2, 5, -4, 4, -3, -4, 4, -4, 1, 4, -4, 5, 2, 1, 4, -3, 5, -2, -3, 1, 2, -4, 5, -2, 1, -3, 2, -2, 5, 4, 5, -2, 2, 1, -3, -2, 2, 5, -4, -3, 4, 1, 5, -4, 4, -4, 1, 4, -4, 4, -3, -4, 4, 5, -4, -3, 4, -4, 4, -4, 4, 1, 5, -4, 4, 1, -4, 4, -4, 5, 4, 1, -4, 4, -4, 4, -3, -4, 4, -4, 5, -3, 4, 1, -4, 2, 5]

The function `dynamics` returns data on the collisions that have occurred.

In [83]:
function draw_disc!(x, y, r)
    θs = [0:0.1:2π; 0]
    xs = [x + r*cos(θ) for θ in θs]
    ys = [y + r*sin(θ) for θ in θs]
    
    plot!(xs, ys)
end

draw_disc! (generic function with 1 method)

In [84]:
@manipulate for i in slider(1:length(positions), value=1)
    x1, y1, x2, y2 = positions[i]
    
    a = w/2
    b = h/2
    
    plot([-a, a, a, -a, -a], [-b, -b, b, b, -b], aspectratio=1, leg=false, xlim=(-0.75,0.75), ylim=(-0.5,0.5))
    draw_disc!(x1, y1, r) 
    draw_disc!(x2, y2, r) 
end

Interact.Slider{Int64}(5: "input-2" = 1 Int64 , "", 1, 1:1000000, "horizontal", true, "d", true)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.75 
 
 
 -0.50 
 
 
 -0.25 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 -0.50 
 
 
 -0.25 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50

# Hopping

The data from the simulation directly gives disc and wall collision times.
The other times that we require are hopping times, both horizontal, when $x_1 = x_2$, and vertical, when $y_1 = y_2$. 

Vertical hops can be detected from the data when $y_2 - y_1$ changes sign between two consecutive collisions.
Using the relative velocity $v_2 - v_1$, the vertical hopping time when $y_2 - y_1 = 0$ can be recovered.

In [85]:
"""
Calculate the times at which horizontal hops occur
"""
function horizontal_hopping_times(times, positions, velocities)
    Δxs = [x[3] - x[1] for x in positions]  # x_2 - x_1
    Δus = [v[3] - v[1] for v in velocities]  # u_2 - u_1;
    
    # indices where there is a hop between collisions i and i+1: 
    horiz_hop_indices = find( sign(Δxs[i]) != sign(Δxs[i+1]) for i in 1:length(positions)-1 );  
    
    # x + t*u = 0   so   t = -x/u
    horiz_hopping_times = times[horiz_hop_indices] - (Δxs[horiz_hop_indices] ./ Δus[horiz_hop_indices])
    
    return horiz_hopping_times
end
## Verified by karel, it is correct.
 

"""
Calculate the times at which horizontal hops occur
"""
function vertical_hopping_times(times, positions, velocities)
    Δys = [x[4] - x[2] for x in positions]  # y_2 - y_1
    Δvs = [v[4] - v[2] for v in velocities]  # v_2 - v_1;
    
    # indices where there is a hop between collisions i and i+1: 
    vert_hop_indices = find( sign(Δys[i]) != sign(Δys[i+1]) for i in 1:length(positions)-1 );  
    
    # x + t*u = 0   so   t = -x/u
    vert_hopping_times = times[vert_hop_indices] - (Δys[vert_hop_indices] ./ Δvs[vert_hop_indices])
    
    return vert_hopping_times
end

vertical_hopping_times

 ## Analytical Expressions
 
 ### 1. Volume
 
 The Volume function has three regimenes. First, when all hopping is possible, that is, (convention $w>h$) $r<h/4$. Then, horizontal hopping is excluded, but vertical still posible, i.e. $h/4\leq r < w/4$. At last, no hopping is possible, that means, $w/4\leq r \leq r_{max}$. 
 
 The Excluded Volume is evaluated in two parts: the round part and the triangular part. We use the notation "Neg" in the functions to indicate that is the excluded volume, "free" to indicate all hoping is posible, "vhopp" to indicate only vertical hopping is possible, and "nohopp" to indicate no hopping is possible.

In [86]:
cte=3π/2 #E=1/2, -> abs(v^2)=1 as above
rhmax= h/4 #maximal radius in which a horizontal hopp is possible
rvmax= w/4 #maximal radius in which a vertical hopp is possible (any hopp is not posible in our geometry after this)
rmax=  (h+w-√(2*h*w))/2   #maximal possible radius for discs


volBox(a,b)=16*a^2*b^2
volNegfree(a,b,r)=16*π*a*b*r^2-64/3*r^3*(a+b)+8*r^4

expresioncircularvol(a,b,r,θ)=32*r^2*a*b*θ +64/3*r^3*(a*cos(θ)-b*sin(θ)) + 8*r^4*sin(θ)^2

function volNegCircle(a,b,r)
    # This function only has use if r>rhmax
    α=asin(b/r)
    if(r<rvmax)
        β=0
    else
        β=acos(a/r)
    end
    result=expresioncircularvol(a,b,r,α)-expresioncircularvol(a,b,r,β)
    return result
end

vollowertriangle(a,b,r)= 32*a^2*b*sqrt(r^2-a^2)-32/3*a^2*(r^2-a^2+2*b*sqrt(r^2-a^2))+8*a^2*(r^2-a^2)

voluppertriangle(a,b,r)= 32*b^2*a*sqrt(r^2-b^2)-32/3*b^2*(r^2-b^2+2*a*sqrt(r^2-b^2))+8*b^2*(r^2-b^2)


function volNegGeneral(a,b,r)
    if (r<rhmax)
        result=volNegfree(a,b,r)
    elseif (rhmax<=r) && (r<rvmax)
       # println("a=",a," r =" , r, "r^2-a^2 = ", r^2-a^2)
        result=volNegCircle(a,b,r)+voluppertriangle(a,b,r)
    else
        result=volNegCircle(a,b,r)+voluppertriangle(a,b,r)+vollowertriangle(a,b,r)
    end
    return result
end

function vol(a,b,r) 
    result=volBox(a,b)-volNegGeneral(a,b,r)
  #  print(" vol = ", result, " \t ")
    return result
end 

vol (generic function with 1 method)

In [87]:
?||

search:



```
x || y
```

Short-circuiting boolean OR.


In [88]:
area_vert_hopp(a,b,r)=16*√(2)*b*(r-a)^2 #This is right, but SymPy has problems refining Heaviside function and DiracDelta
area_horiz_hopp(a,b,r)=16*√(2)*a*(r-b)^2 # The Area HAS TWO SIDES. This is weird, but true.
#Those two don't generalize for larger r--- they just stop making sense.

area_horiz_hopp (generic function with 1 method)

In [89]:
area_wall_free(a,b,r)=8*a*b^2-4*π*b*r^2+16/3*r^3 #perfect but needs generalization

exprpos_wall(a,b,r)=8*a*b^2
exprcircular_wall(a,b,r,θ)=8*b*r^2*θ+16/3*r^3*(cos(θ)) #sector

# la parte del "upper triangle"
a_wall_uppt(a,b,r)=8/3*b^2*sqrt(r^2-b^2)
a_wall_lowt(a,b,r)=8/3*a^2*sqrt(r^2-a^2)

function area_wall(a,b,r)
    if (r<rhmax)
        result=area_wall_free(a,b,r)
    elseif (rhmax<=r) && (r<rvmax)
         α=asin(b/r)
         negativo=exprcircular_wall(a,b,r,α)-exprcircular_wall(a,b,r,0)+a_wall_uppt(a,b,r)
         result=(exprpos_wall(a,b,r)-negativo)*2
    else
        α=asin(b/r)
        β=acos(a/r)
        negativo=0 # (exprcircular_wall(a,b,r,α)-exprcircular_wall(a,b,r,β))+a_wall_uppt(a,b,r)+a_wall_lowt(a,b,r)
        result=(exprpos_wall(a,b,r)-negativo)*4
    end
    return result
end

area_wall (generic function with 1 method)

In [90]:
area_col_free(a,b,r)=(16*π*a*b*r-32*(a+b)*r^2+16*r^3)*√(2) 
exprcircular_col(a,b,r,θ)=16*√2*(2*a*b*r*θ + 2*r^2*(a*cos(θ)-b*sin(θ)) + r^3*sin(θ)^2 )

function area_col(a,b,r)
    if (r<rhmax)
        result=area_col_free(a,b,r)
    elseif (rhmax<=r) && (r<rvmax)
        α=asin(b/r)
        result=exprcircular_col(a,b,r,α)-exprcircular_col(a,b,r,0)
    else
        α=asin(b/r)
        β=acos(a/r)        
        result=exprcircular_col(a,b,r,α)-exprcircular_col(a,b,r,β)   
    end
  #  print(" acol= ",result,  " \n ")
    return result
end

area_col (generic function with 1 method)

### 3. Mean Times Machta Zwanzig

In [91]:
horiz_hop_analytical(a,b,r)=cte*vol(a,b,r)/area_horiz_hopp(a,b,r)
vert_hop_analytical(a,b,r)=cte*vol(a,b,r)/area_vert_hopp(a,b,r)
right_wall_analytical(a,b,r)=cte*vol(a,b,r)/area_wall(a,b,r) #perfect
disc_collision_analytical(a,b,r)=cte*vol(a,b,r)/area_col(a,b,r)

horiz_hop_analytical_d(a, b, r) = (3π / (2*√(2))) * (2*a^2*b^2 - 2π*a*b*r^2 + ((a+b)/3)*(2*r)^3 - r^4) / (a*√(2)*(b-r)^2)
disc_collision_analytical_d(a, b, r) = (3π / (2*√(2))) * (2*a^2*b^2 - 2π*a*b*r^2 + ((a+b)/3)*(2*r)^3 - r^4) / (2π*a*b*r - 4*(a+b)*r^2 + 2r^3 )

disc_collision_analytical_d (generic function with 1 method)

In [93]:
function simulation(w, h, num_collisions=10^7)
    
    num_horiz_hop_data=Float64[]
    exact_horiz_hop_data=Float64[]
    
    num_vert_hop_data=Float64[]
    exact_vert_hop_data=Float64[]
    
    num_disc_collision_data = Float64[]
    exact_disc_collision_data = Float64[]
    
    num_rightwall_data = Float64[]
    exact_rightwall_data = Float64[]
    
    
    
    exhopd = Float64[]
    excold = Float64[]
    
    
    rs = 0.005:0.005:rmax
    
    for r in rs
        print(r, " ")
        
        a = w/2 - r
        b = h/2 - r
    
        times, positions, velocities, collision_types = dynamics(w, h, r, num_collisions);
        
        
        horiz_hopping_times = horizontal_hopping_times(times, positions, velocities)

        vert_hopping_times = vertical_hopping_times(times, positions, velocities)

        if(r<rhmax)
            push!(num_vert_hop_data, mean(diff(vert_hopping_times)))   # diff gives inter-hop times
            push!(exact_vert_hop_data, vert_hop_analytical(a, b, r))
        end
        
        push!(num_horiz_hop_data, mean(diff(horiz_hopping_times)))   # diff gives inter-hop times
        push!(exact_horiz_hop_data, horiz_hop_analytical(a, b, r))
        push!(exhopd, horiz_hop_analytical_d(a, b, r))
        
        
        disc_collision_times = times[collision_types .== 5]
        push!(num_disc_collision_data, mean(diff(disc_collision_times)))   # diff gives inter-hop times
        push!(exact_disc_collision_data, disc_collision_analytical(a, b, r))
        push!(excold, disc_collision_analytical_d(a, b, r))
    
        rightwall_times = times[collision_types .== 1]
        push!(num_rightwall_data, mean(diff(rightwall_times)))   # diff gives inter-hop times
        push!(exact_rightwall_data, right_wall_analytical(a, b, r))
        
    end
    
    return (rs,  num_vert_hop_data, exact_vert_hop_data, num_horiz_hop_data, exact_horiz_hop_data, num_disc_collision_data, 
        exact_disc_collision_data, num_rightwall_data, exact_rightwall_data, excold, exhopd)
                
    end

simulation (generic function with 2 methods)

In [94]:
w, h = 1.5, 1.0
(rs, num_vert_hop_data, exact_vert_hop_data,  num_horiz_hop_data, exact_horiz_hop_data, num_disc_collision_data, 
    exact_disc_collision_data, num_rightwall_data, exact_rightwalldata, excold, exhopd)= simulation(w, h);

0.005 0.01 0.015 0.02 0.025 0.03 0.035 0.04 0.045 0.05 0.055 0.06 0.065 0.07 0.075 0.08 0.085 0.09 0.095 0.1 0.105 0.11 0.115 0.12 0.125 0.13 0.135 0.14 0.145 0.15 0.155 0.16 0.165 0.17 0.17500000000000002 0.18 0.185 0.19 0.195 0.2 0.20500000000000002 0.21 0.215 0.22 0.225 0.23 0.23500000000000001 0.24 0.245 0.25 0.255 0.26 0.265 0.27 0.275 0.28 0.28500000000000003 0.29 0.295 0.3 0.305 0.31 0.315 0.32 0.325 0.33 0.335 0.34 0.34500000000000003 0.35000000000000003 0.355 0.36 0.365 0.37 0.375 0.38 

In [95]:
using JLD

INFO: Recompiling stale cache file /home/karel/.julia/lib/v0.6/HDF5.ji for module HDF5.
INFO: Recompiling stale cache file /home/karel/.julia/lib/v0.6/JLD.ji for module JLD.


In [99]:
save("DatosNumericos10a7.jld", "rs", rs, 
    "num_vert_hop_data", num_vert_hop_data,
    "num_horiz_hop_data", num_horiz_hop_data,
    "num_disc_collision_data", num_disc_collision_data,
    "num_rightwall_data", num_rightwall_data
)

In [98]:
Plots.scatter(rs, num_rightwall_data, m=:square, label="numerical") #it still needs the fucken line
plot!(rs, exact_rightwalldata, label="exact", linewidth=2) #absolut PERFECT

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 numerical 
 
 
 
 exact

In [25]:
Plots.scatter(rs, num_disc_collision_data, m=:square, label="numerical")
plot!(rs, exact_disc_collision_data, label="exact")
plot!(rs, excold, label="DavEx")
plot!(rs, zeros(rs), lw=2)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.24 
 
 
 0.26 
 
 
 0.28 
 
 
 0.30 
 
 
 0.32 
 
 
 0.34 
 
 
 0.36 
 
 
 -20 
 
 
 -15 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 numerical 
 
 
 
 exact 
 
 
 
 DavEx 
 
 
 
 y4

In [27]:
rhmax, rvmax, rmax

(0.25, 0.375, 0.3839745962155614)

In [94]:
#rsredux=rs[find( r<rhmax ) in rs]
rsredux=filter(r->r<rhmax, rs)
plot(rsredux, num_vert_hop_data, m=:square, label="numerical")
plot!(rsredux, exact_vert_hop_data, label="exact") 
#plot!(rs, exhopd, label="DavidEx")
#Formula David, raiz de dos muy grande 

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.225 
 
 
 0.230 
 
 
 0.235 
 
 
 0.240 
 
 
 0.245 
 
 
 1.05 
 
 
 1.08 
 
 
 1.11 
 
 
 1.14 
 
 
 1.17 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 numerical 
 
 
 
 exact

In [25]:
exact_vert_hop_data

49-element Array{Float64,1}:
 1.67143
 1.67637
 1.68087
 1.68493
 1.68853
 1.69164
 1.69427
 1.69638
 1.69795
 1.69898
 1.69943
 1.69929
 1.69853
 ⋮      
 1.39395
 1.36665
 1.33805
 1.30817
 1.27711
 1.24497
 1.21191
 1.17815
 1.144  
 1.10984
 1.07623
 1.04385

In [85]:
plot(rs, exact_horiz_hop_data./num_horiz_hop_data, m=:square)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 0.25 
 
 
 0.95 
 
 
 1.00 
 
 
 1.05 
 
 
 1.10 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [86]:
w,h,r=1,1.5,0.16
num_collisions=100
times, positions, velocities, collision_types = dynamics(w, h, r, num_collisions);
horiz_hopping_times = horizontal_hopping_times(times, positions, velocities)

24-element Array{Float64,1}:
  2.15516
  2.26523
  6.53391
  7.07854
  8.31134
  8.8148 
 10.2538 
 11.6928 
 12.7251 
 13.0481 
 13.7237 
 14.7224 
 15.7211 
 16.7197 
 17.7184 
 17.7848 
 18.7519 
 20.2571 
 21.6495 
 22.9693 
 27.9883 
 29.1532 
 30.267  
 31.3802 

In [87]:
histogram(diff(horiz_hopping_times))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 0 
 
 
 3 
 
 
 6 
 
 
 9 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [16]:
histogram(diff(horiz_hopping_times), normed=true)

LoadError: [91mUndefVarError: horiz_hopping_times not defined[39m

# Hitting right wall

In [29]:
V_exact(a, b, r) = 16a^2*b^2 - 16π*a*b*r^2 + (64/3)*(a+b)*r^3 - 8r^4

disc_1_hits_right_wall(a, b, r) = 8*a*b^2 - 4π*b*r^2 + (16/3) * r^3

disc_1_hits_right_wall (generic function with 1 method)

In [21]:
disc_1_hits_right_wall_times = times[collision_types .== 1]

8444-element Array{Float64,1}:
    1.98975
    2.95109
    3.19826
    3.27112
    3.75055
    3.8023 
    3.85744
    4.60162
    5.45908
    6.48873
    6.76875
    7.20229
    7.24794
    ⋮      
 8195.99   
 8196.62   
 8196.99   
 8197.31   
 8197.96   
 8205.74   
 8207.99   
 8208.16   
 8208.29   
 8209.39   
 8210.45   
 8210.82   

In [22]:
S = 2*π^2 
B = 4π / 3

factor = S / B

exact_mean_hitting_time(a, b, r) = factor * V_exact(a, b, r) / disc_1_hits_right_wall(a, b, r)

exact_mean_hitting_time (generic function with 1 method)

In [23]:
function simulation2(w, h, num_collisions=10^5)
    
    num_hitting_times = Float64[]
    exact_hitting_times = Float64[]
    
    rs = 0.005:0.005:0.23
    
    for r in rs
        print(r, " ")
        
        a = w/2 - r
        b = h/2 - r
    
        times, positions, velocities, collision_types = dynamics(w, h, r, num_collisions);
        
        disc_1_hits_right_wall_times = times[collision_types .== 1]

        push!(num_hitting_times, mean(diff(disc_1_hits_right_wall_times)))   # diff gives inter-hop times
        push!(exact_hitting_times, exact_mean_hitting_time(a, b, r))
        
        
    end
    
    return rs, num_hitting_times, exact_hitting_times
end

simulation2 (generic function with 2 methods)

In [24]:
rs, num_hitting_times, exact_hitting_times = simulation2(1.0, 1.0)

0.005 0.01 0.015 0.02 0.025 0.03 0.035 0.04 0.045 0.05 0.055 0.06 0.065 0.07 0.075 0.08 0.085 0.09 0.095 0.1 0.105 0.11 0.115 0.12 0.125 0.13 0.135 0.14 0.145 0.15 0.155 0.16 0.165 0.17 0.175 0.18 0.185 0.19 0.195 0.2 0.205 0.21 0.215 0.22 0.225 0.23 

(0.005:0.005:0.23, [4.67758, 4.58948, 4.49672, 4.43488, 4.44265, 4.41639, 4.36844, 4.30181, 4.20581, 4.21032  …  1.95438, 1.88366, 1.7964, 1.68128, 1.57659, 1.44244, 1.33599, 1.30018, 1.06917, 1.0123], [4.66453, 4.6152, 4.56441, 4.51217, 4.4585, 4.4034, 4.34688, 4.28895, 4.22961, 4.16888  …  1.98724, 1.88377, 1.77828, 1.67073, 1.56107, 1.44932, 1.33552, 1.21988, 1.10282, 0.985235])

In [25]:
plot(rs, num_hitting_times, m=:square, label="numerical")
plot!(rs, exact_hitting_times, m=:square, label="exact")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 numerical 
 
 
 
 
 
 exact

# Disc collisions

In [29]:
disc_collision_exact_area(a, b, r) = 16*π*a*b*r - 32*(a+b)*r^2 + 16*r^3 

exact_disc_collision(a, b, r) = factor * V_exact(a, b, r) / disc_collision_exact_area(a, b, r)

exact_disc_collision (generic function with 1 method)

In [30]:
function simulation_disc_collisions(w, h, num_collisions=10^5)
    
    num_times = Float64[]
    exact_times = Float64[]
    
    rs = 0.005:0.005:0.23
    
    for r in rs
        print(r, " ")
        
        a = w/2 - r
        b = h/2 - r
    
        times, positions, velocities, collision_types = dynamics(w, h, r, num_collisions);
        
        collision_times = times[collision_types .== 5]

        push!(num_times, mean(diff(collision_times)))   # diff gives inter-hop times
        push!(exact_times, exact_disc_collision(a, b, r))
        
        
    end
    
    return rs, num_times, exact_times
end

simulation_disc_collisions (generic function with 2 methods)

In [31]:
rs, num_disc_collision_times, exact_disc_collision_times = simulation_disc_collisions(w, h)

0.005 0.01 0.015 0.02 0.025 0.03 0.035 0.04 0.045 0.05 0.055 0.06 0.065 0.07 0.075 0.08 0.085 0.09 0.095 0.1 0.105 0.11 0.115 0.12 0.125 0.13 0.135 0.14 0.145 0.15 0.155 0.16 0.165 0.17 0.175 0.18 0.185 0.19 0.195 0.2 0.205 0.21 0.215 0.22 0.225 0.23 

(0.005:0.005:0.23, [52.3012, 25.9111, 17.0541, 12.5565, 10.0964, 8.41752, 7.11017, 6.0247, 5.34809, 4.79019  …  0.62033, 0.577382, 0.523786, 0.478765, 0.435009, 0.395654, 0.353614, 0.319724, 0.277916, 0.243282], [74.4391, 36.9232, 24.4074, 18.1415, 14.3756, 11.8597, 10.058, 8.70279, 7.64516, 6.79585  …  0.882501, 0.812518, 0.745294, 0.680651, 0.618443, 0.558563, 0.50095, 0.445609, 0.392653, 0.342378])

In [33]:
plot(rs, num_disc_collision_times, m=:square, label="numerical")
plot!(rs, exact_disc_collision_times, m=:square, label="exact")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 numerical 
 
 
 
 
 
 exact

In [35]:
plot(rs, exact_disc_collision_times./num_disc_collision_times, m=:square, label="ratio")


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 1.40 
 
 
 1.41 
 
 
 1.42 
 
 
 1.43 
 
 
 1.44 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ratio

Something to do with relative velocity?

## Vertical hopping

In [35]:
A_vert_hop_exact(a, b, r) = 8*b * sqrt(2)* (a-r)^2 

S = 2*π^2 
B = 4π / 3

factor = S / B

exact_vert_hopping(a, b, r) = factor * V_exact(a, b, r) / (2 * A_vert_hop_exact(a, b, r))
# el factor de 2 se debe a que se puede acceder desde dos lados al conjunto

exact_vert_hopping (generic function with 1 method)

In [36]:
function simulation_vert_hopping(w, h, num_collisions=10^5)
    
    num_times = Float64[]
    exact_times = Float64[]
    
    rs = 0.005:0.005:0.23
    
    for r in rs
        print(r, " ")
        
        a = w/2 - r
        b = h/2 - r
    
        times, positions, velocities, collision_types = dynamics(w, h, r, num_collisions);
        
        vert_hopping_times = vertical_hopping_times(times, positions, velocities)

        push!(num_times, mean(diff(vert_hopping_times)))   # diff gives inter-hop times
        push!(exact_times, exact_vert_hopping(a, b, r))
        
    end
    
    return rs, num_times, exact_times
end

simulation_vert_hopping (generic function with 2 methods)

In [37]:
rs, num_vert_hopping_times, exact_vert_hopping_times = simulation_vert_hopping(w, h)

0.005 0.01 0.015 0.02 0.025 0.03 0.035 0.04 0.045 0.05 0.055 0.06 0.065 0.07 0.075 0.08 0.085 0.09 0.095 0.1 0.105 0.11 0.115 0.12 0.125 0.13 0.135 0.14 0.145 0.15 0.155 0.16 0.165 0.17 0.175 0.18 0.185 0.19 0.195 0.2 0.205 0.21 0.215 0.22 0.225 0.23 

(0.005:0.005:0.23, [1.70211, 1.70849, 1.73376, 1.7546, 1.75107, 1.75934, 1.79558, 1.80619, 1.78078, 1.82043  …  2.46747, 2.46998, 2.53552, 2.69197, 2.65635, 2.87706, 3.05583, 3.35446, 3.65881, 4.35129], [1.68272, 1.69931, 1.71586, 1.73238, 1.74886, 1.76531, 1.78174, 1.79815, 1.81456, 1.83096  …  2.44723, 2.50425, 2.57218, 2.65492, 2.75835, 2.89172, 3.07038, 3.32167, 3.69898, 4.32082])

In [38]:
plot(rs, num_vert_hopping_times, m=:square, label="numerical")
plot!(rs, exact_vert_hopping_times, m=:square, label="exact")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 2.0 
 
 
 2.5 
 
 
 3.0 
 
 
 3.5 
 
 
 4.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 numerical 
 
 
 
 
 
 exact

El factor de 2 viene del hecho de que se puede acceder desde dos lados.

In [39]:
plot(rs, num_vert_hopping_times ./ exact_vert_hopping_times, m=:square, label="numerical")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 0.97 
 
 
 0.98 
 
 
 0.99 
 
 
 1.00 
 
 
 1.01 
 
 
 1.02 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 numerical

## Symbolics

In [8]:
using SymPy

In [10]:
@syms x y

(x, y)

  likely near /Users/dpsanders/.julia/v0.6/IJulia/src/kernel.jl:31
  likely near /Users/dpsanders/.julia/v0.6/IJulia/src/kernel.jl:31
  likely near /Users/dpsanders/.julia/v0.6/IJulia/src/kernel.jl:31
in jprint at /Users/dpsanders/.julia/v0.6/SymPy/src/display.jl


In [11]:
x

x

In [12]:
x*x

 2
x 

In [13]:
sin(x)

sin(x)

In [16]:
simplify(sin(x)^2 + cos(x)^2)

1